In [3]:
import urllib.request #ネットからファイルをダウンロードする標準ライブラリ
import os

#cat
cat_url = "https://storage.googleapis.com/quickdraw_dataset/full/raw/cat.ndjson"
cat_path = "/content/cat.ndjson"

urllib.request.urlretrieve(cat_url, cat_path) #urlからファイルをローカルに保存
print("cat.ndjson ダウンロード完了:", cat_path)
print("ファイルが存在するか:", os.path.exists(cat_path))#保存できたか確認

#apple
apple_url = "https://storage.googleapis.com/quickdraw_dataset/full/raw/apple.ndjson"
apple_path = "/content/apple.ndjson"

urllib.request.urlretrieve(apple_url, apple_path)
print("apple.ndjson ダウンロード完了:", apple_path)
print("ファイルが存在するか:", os.path.exists(apple_path))


cat.ndjson ダウンロード完了: /content/cat.ndjson
ファイルが存在するか: True
apple.ndjson ダウンロード完了: /content/apple.ndjson
ファイルが存在するか: True


In [4]:
import json #jsonを読み込む
import numpy as np
import tensorflow as tf #LSTMつかうため
from sklearn.model_selection import train_test_split

#ストローク形式の描画データ(strokes)を系列データ(seq)に変換
def convert_strokes_to_seq(strokes):
    seq = []
    prev_x, prev_y = 0, 0 #seq(前の点の座標)初期化
    for stroke in strokes: #[[x1, x2...], [y1, y2...]]
        x_list, y_list = stroke[:2] #strokeは[x座標リスト, y座標リスト]それを2つに分ける

        for i in range(len(x_list)):
            x, y = x_list[i], y_list[i] #現在の(x,y)座標を取得
            dx, dy = x - prev_x, y - prev_y #前の点との座標差分(Δx,Δy)

           #ペンの状態をone-hotで、
            pen = [1, 0, 0] #ペンを動かしている
            if i == len(x_list) - 1: #最後の点か判定(最終点->ペンを離す状態に切り替える)
                pen = [0, 1, 0]  #ペンを離す
            seq.append([dx, dy] + pen) #差分とペン状態をまとめて1つのベクトルとしてseqに追加
            prev_x, prev_y = x, y #今の点を次の点に設定

    seq.append([0, 0, 0, 0, 1])  #終了を示すトークン
    return seq #完成した系列を返す(1つの絵->1つのseqとなる)

#ndjson読み込む。各行を(系列+ラベル)の形に変換
def load_drawings(filepath, label=0, limit=1000):
    data = []
    with open(filepath, "r") as f:
        for i, line in enumerate(f):
            if i >= limit:
                break
            sample = json.loads(line) #各行(json)を辞書に変換
            seq = convert_strokes_to_seq(sample["drawing"]) #drawingキー(ストローク配列)を系列データに変換
            data.append((seq, label)) #(系列、ラベル)のタプルとしてリストに格納
    return data #(sequence,label)のリストを返す

# ファイル読み込み
cat_data = load_drawings("/content/cat.ndjson", label=0)
apple_data = load_drawings("/content/apple.ndjson", label=1)
all_data = cat_data + apple_data #1つのリストに結合(2クラス分類用データ)

#パディングと分割
MAXLEN = 200 #LSTM固定長必要
#各系列の長さをMAX200(それ以上は切る、足りない部分は0パディング)
X = [seq[:MAXLEN] + [[0]*5]*(MAXLEN-len(seq)) if len(seq)<MAXLEN else seq[:MAXLEN] for seq, _ in all_data]
y = [label for _, label in all_data] #正解ラベル(0か1)各系列のラベルを取り出してyとする

#numpy配列に変換->tensorflowで使えるように
X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.int32)

#8:2に分割して学習とテスト用データを作成
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#LSTMモデル
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(MAXLEN, 5)), #(200,5)の系列を入力
    tf.keras.layers.LSTM(64), #64ユニットのRNN
    tf.keras.layers.Dense(32, activation='relu'), #32ユニット、ReLu
    tf.keras.layers.Dense(2, activation='softmax')  #2クラス出力
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",  #損失関数
    metrics=["accuracy"]
)

# 学習
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)



Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.6003 - loss: 0.6673 - val_accuracy: 0.7325 - val_loss: 0.5362
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.7771 - loss: 0.4996 - val_accuracy: 0.7950 - val_loss: 0.4949
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8246 - loss: 0.4427 - val_accuracy: 0.8475 - val_loss: 0.3894
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.8514 - loss: 0.3860 - val_accuracy: 0.8650 - val_loss: 0.3631
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8459 - loss: 0.3901 - val_accuracy: 0.8550 - val_loss: 0.3530
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8347 - loss: 0.3746 - val_accuracy: 0.8550 - val_loss: 0.3517
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8532 - loss: 0.3437 - val_accuracy: 0.7325 - val_loss: 0.4495
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6786 - loss: 0.5247 - val_accuracy: 0.7075 - v

In [6]:
from IPython.display import display, HTML

canvas_code = """
<canvas id="canvas" width="500" height="500" style="border:1px solid black;"></canvas><br>
<button onclick="clearCanvas()">🧹 クリア</button>
<button onclick="sendData()">📤 送信して分類</button>
<p id="output"></p>

<script>
let canvas = document.getElementById("canvas");
let ctx = canvas.getContext("2d");
let drawing = false;
let strokes = [];
let currentStroke = [];

canvas.addEventListener("mousedown", (e) => {
  drawing = true;
  currentStroke = [];
  draw(e);
});

canvas.addEventListener("mouseup", () => {
  drawing = false;
  if (currentStroke.length > 0) {
    strokes.push(currentStroke);
  }
  ctx.beginPath();
});

canvas.addEventListener("mousemove", draw);

function draw(e) {
  if (!drawing) return;
  const rect = canvas.getBoundingClientRect();
  const x = e.clientX - rect.left;
  const y = e.clientY - rect.top;

  ctx.lineWidth = 6;
  ctx.lineCap = "round";
  ctx.strokeStyle = "black";

  ctx.lineTo(x, y);
  ctx.stroke();
  ctx.beginPath();
  ctx.moveTo(x, y);

  currentStroke.push([x, y]); // ← 座標を記録
}

function clearCanvas() {
  ctx.clearRect(0, 0, canvas.width, canvas.height);
  strokes = [];
  currentStroke = [];
  document.getElementById("output").innerText = "";
}

function sendData() {
  if (currentStroke.length > 0) {
    strokes.push(currentStroke);
    currentStroke = [];
  }
  google.colab.kernel.invokeFunction('classify_strokes', [strokes], {});
}
</script>
"""
display(HTML(canvas_code))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
予測: 😺 cat（信頼度: 0.72）
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
予測: 🍎 apple（信頼度: 0.91）
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
予測: 🍎 apple（信頼度: 0.91）
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
予測: 🍎 apple（信頼度: 0.69）
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
予測: 🍎 apple（信頼度: 0.91）
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
予測: 😺 cat（信頼度: 0.93）
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
予測: 😺 cat（信頼度: 0.97）


In [7]:
from google.colab import output
#JavaScriptからPython関数を呼び出すためのモジュール
import numpy as np

class_names = ["cat", "apple"] #リスト

#JSからくる系列をLSTM用の数値系列に変換
def preprocess_strokes(raw_strokes, maxlen=200):
    seq = []
    prev_x, prev_y = 0, 0

    for stroke in raw_strokes:
        for i, point in enumerate(stroke):
            x, y = point
            dx = x - prev_x
            dy = y - prev_y
            pen = [1, 0, 0]
            if i == len(stroke) - 1:
                pen = [0, 1, 0]
            seq.append([dx, dy] + pen)
            prev_x, prev_y = x, y
    seq.append([0, 0, 0, 0, 1])

    #パディングと分割
    if len(seq) < maxlen:
        seq += [[0]*5]*(maxlen - len(seq))
    else:
        seq = seq[:maxlen]
    return np.expand_dims(np.array(seq, dtype=np.float32), axis=0)

#キャンバスからきたストローク系列を受け取り、分類
def classify_strokes(js_strokes):
    x = preprocess_strokes(js_strokes) #ストロークをLSTM入力形式に変換
    pred = model.predict(x) #予測
    label_id = np.argmax(pred[0]) #最大値のインデックス(0か1)を取得
    confidence = np.max(pred[0])
    label = "cat" if label_id == 0 else "apple"
    print(f"予測: {label}（信頼度: {confidence:.2f}）")

#JSからの呼び出し
output.register_callback('classify_strokes', classify_strokes)